# Neural networks

Principle : In this section we'll focus on one specific type of Deep Learning algorithm, namely multilayer perceptrons. MLPs can be viewed as generalizations of linear models that perform multiple stages of processing to come to a decision.
This model has a lot more coefficients (also called weights) to learn: there is one between every input and every hidden unit (which make up the hidden layer), and one between every unit in the hidden layer and the output.
After computing a weighted sum for each hidden unit, a nonlinear function is applied to the result, usually the rectifying nonlinearity (also known as rectified linear unit or relu) or the tangens hyperbolicus (tanh). The relu cuts off values below zero, while tanh saturates to –1 for low input values and +1 for high input values

In [1]:
%load_ext autoreload
%autoreload
from utils import feature_selection, PCA_reduction, perf, time_comparison

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, Normalizer

### First test

In [7]:
gt = pd.read_csv('../dumps/2020.01.13-14.25.csv')
cols = [col for col in gt.columns if col not in ['label']]
data = gt[cols]
target = gt['label']

data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.20, random_state = 0)
scaler = Normalizer()
scaler.fit(data_train)
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)

mlp = MLPClassifier(solver='lbfgs', random_state=0, max_iter=10000)
mlp.fit(data_train, target_train)
print("Accuracy on training set: {:.3f}".format(mlp.score(data_train, target_train))) 
print("Accuracy on test set: {:.3f}".format(mlp.score(data_test, target_test)))

Accuracy on training set: 0.981
Accuracy on test set: 0.754


Considering the results, we rather be overfitting.

### Further tests

In [8]:
gt = pd.read_csv('../dumps/2020.02.10-12.14.csv')
cols = [col for col in gt.columns if col not in ['label']]
data = gt[cols]
target = gt['label']

data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.20, random_state = 0)
scaler = Normalizer()
scaler.fit(data_train)
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)

#### Solver

The solver for weight optimization.
- ‘lbfgs’ is an optimizer in the family of quasi-Newton methods.
- ‘sgd’ refers to stochastic gradient descent.
- ‘adam’ refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy Ba

Note: The default solver ‘adam’ works pretty well on relatively large datasets (with thousands of training samples or more) in terms of both training time and validation score. For small datasets, however, ‘lbfgs’ can converge faster and perform better.

In [9]:
solver = ['lbfgs','sgd', 'adam']
for i in solver:
    print("Solver : %s" % i)
    mlp = MLPClassifier(solver=i, random_state=0, max_iter=10000) 
    mlp.fit(data_train, target_train)
    print("Accuracy on training set: {:.3f}".format(mlp.score(data_train, target_train))) 
    print("Accuracy on test set: {:.3f}".format(mlp.score(data_test, target_test)))

Solver : lbfgs
Accuracy on training set: 0.993
Accuracy on test set: 0.865
Solver : sgd
Accuracy on training set: 0.895
Accuracy on test set: 0.900
Solver : adam
Accuracy on training set: 0.977
Accuracy on test set: 0.873


As we could expect, the 'adam' algorithm performs quite well on our dataset. Still, the performance on the test set might be improved by tuning other parameters.

#### Activation

Activation function for the hidden layer.
- ‘identity’, no-op activation, useful to implement linear bottleneck, returns f(x) = x
- ‘logistic’, the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).
- ‘tanh’, the hyperbolic tan function, returns f(x) = tanh(x).
- ‘relu’, the rectified linear unit function, returns f(x) = max(0, x)

In [10]:
act = ['identity','logistic', 'tanh', 'relu']
for i in act:
    print("function : %s" % i)
    mlp = MLPClassifier(activation=i, random_state=0, max_iter=10000) 
    mlp.fit(data_train, target_train)
    print("Accuracy on training set: {:.3f}".format(mlp.score(data_train, target_train))) 
    print("Accuracy on test set: {:.3f}".format(mlp.score(data_test, target_test)))

function : identity
Accuracy on training set: 0.895
Accuracy on test set: 0.900
function : logistic
Accuracy on training set: 0.895
Accuracy on test set: 0.900
function : tanh
Accuracy on training set: 0.895
Accuracy on test set: 0.900
function : relu
Accuracy on training set: 0.977
Accuracy on test set: 0.873


In this case, the 'identity' activation didn't manage to make the algorithm converge. For the others, both 'logistic' and 'tanh' provided sam results and performed better on the test set than the training set, which is the opposite for the 'relu' activation.

#### Learning rate

Learning rate schedule for weight updates (only for 'sgd' solver).
- ‘constant’ is a constant learning rate given by ‘learning_rate_init’.
- ‘invscaling’ gradually decreases the learning rate at each time step ‘t’ using an inverse scaling exponent of ‘power_t’. effective_learning_rate = learning_rate_init / pow(t, power_t)
- ‘adaptive’ keeps the learning rate constant to ‘learning_rate_init’ as long as training loss keeps decreasing. Each time two consecutive epochs fail to decrease training loss by at least tol, or fail to increase validation score by at least tol if ‘early_stopping’ is on, the current learning rate is divided by 5.

In [11]:
learning_rate = ['constant','invscaling', 'adaptive']
for i in learning_rate:
    print("function : %s" % i)
    mlp = MLPClassifier(solver='sgd', learning_rate=i, random_state=0, max_iter=10000) 
    mlp.fit(data_train, target_train)
    print("Accuracy on training set: {:.3f}".format(mlp.score(data_train, target_train))) 
    print("Accuracy on test set: {:.3f}".format(mlp.score(data_test, target_test)))

function : constant
Accuracy on training set: 0.895
Accuracy on test set: 0.900
function : invscaling
Accuracy on training set: 0.895
Accuracy on test set: 0.900
function : adaptive
Accuracy on training set: 0.895
Accuracy on test set: 0.900


For the 'sgd' solver, the learning rate doesn't really matter.

#### Alpha

L2 penalty (regularization term) parameter.

In [12]:
alpha = [0.0001,0.001,0.1,1,10,100,1000]
for i in alpha:
    print("alpha : %s" % i)
    mlp = MLPClassifier(alpha=i, random_state=0, max_iter=10000) 
    mlp.fit(data_train, target_train)
    print("Accuracy on training set: {:.3f}".format(mlp.score(data_train, target_train))) 
    print("Accuracy on test set: {:.3f}".format(mlp.score(data_test, target_test)))

alpha : 0.0001
Accuracy on training set: 0.977
Accuracy on test set: 0.873
alpha : 0.001
Accuracy on training set: 0.974
Accuracy on test set: 0.878
alpha : 0.1
Accuracy on training set: 0.903
Accuracy on test set: 0.904
alpha : 1
Accuracy on training set: 0.895
Accuracy on test set: 0.900
alpha : 10
Accuracy on training set: 0.895
Accuracy on test set: 0.900
alpha : 100
Accuracy on training set: 0.895
Accuracy on test set: 0.900
alpha : 1000
Accuracy on training set: 0.895
Accuracy on test set: 0.900


The more we increase the *alpha* value, the more we reduce overfitting.

#### Hidden layers

This parameter allows us to set the number of layers and the number of nodes we wish to have in the Neural Network Classifier. Each element in the tuple represents the number of nodes at the ith position where i is the index of the tuple.

In [13]:
hidden_layers_size = [(50,50,50), (100,100,100), (50,100,50), (100,50,50), (50,50,100), (100,)]
for i in hidden_layers_size:
    print("layer size : %s" % (i,))
    mlp = MLPClassifier(hidden_layer_sizes=i, random_state=0, max_iter=10000) 
    mlp.fit(data_train, target_train)
    print("Accuracy on training set: {:.3f}".format(mlp.score(data_train, target_train))) 
    print("Accuracy on test set: {:.3f}".format(mlp.score(data_test, target_test)))

layer size : (50, 50, 50)
Accuracy on training set: 0.969
Accuracy on test set: 0.868
layer size : (100, 100, 100)
Accuracy on training set: 0.980
Accuracy on test set: 0.858
layer size : (50, 100, 50)
Accuracy on training set: 0.979
Accuracy on test set: 0.870
layer size : (100, 50, 50)
Accuracy on training set: 0.984
Accuracy on test set: 0.877
layer size : (50, 50, 100)
Accuracy on training set: 0.980
Accuracy on test set: 0.870
layer size : (100,)
Accuracy on training set: 0.977
Accuracy on test set: 0.873


### Best match

In [3]:
gt = pd.read_csv('../dumps/2020.02.10-12.14.csv')
cols = [col for col in gt.columns if col not in ['label']]
data = gt[cols]
target = gt['label']
data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.20, random_state = 0)
scaler = Normalizer()
scaler.fit(data_train)
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)

In [ ]:
parameters = {'solver': ['lbfgs','sgd','adam'], 'max_iter': [1000,10000], 'alpha': [0.0001,0.001,0.1,1,10,100], 'hidden_layer_sizes':[(50,50,50), (100,100,100), (50,100,50), (100,50,50), (50,50,100), (100,)], 'activation':['identity','logistic', 'tanh', 'relu']}
clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
clf.fit(data_train, target_train)
print(clf.score(data_train, target_train))
print(clf.best_params_)

In [ ]:
parameters = {'solver': ['lbfgs','sgd','adam'], 'max_iter': [1000,10000], 'alpha': [0.0001,0.001,0.1,1,10,100], 'hidden_layer_sizes':[(50,50,50), (100,100,100), (50,100,50), (100,50,50), (50,50,100), (100,)], 'activation':['identity','logistic', 'tanh', 'relu']}
clf = RandomizedSearchCV(MLPClassifier(), parameters, n_jobs=-1)
clf.fit(data_train, target_train)
print(clf.score(data_train, target_train))
print(clf.best_params_)

In [4]:
mlp = MLPClassifier(solver='adam',activation='tanh',alpha=100,hidden_layer_sizes=(50, 50, 100))
mlp.fit(data_train, target_train)
print("Accuracy on training set: {:.3f}".format(mlp.score(data_train, target_train))) 
print("Accuracy on test set: {:.3f}".format(mlp.score(data_test, target_test)))

Accuracy on training set: 0.895
Accuracy on test set: 0.900


In [5]:
mlp = MLPClassifier(solver='sgd',activation='tanh',alpha=0.1,hidden_layer_sizes=(100,50,50),max_iter=1000)
mlp.fit(data_train, target_train)
print("Accuracy on training set: {:.3f}".format(mlp.score(data_train, target_train))) 
print("Accuracy on test set: {:.3f}".format(mlp.score(data_test, target_test)))

Accuracy on training set: 0.895
Accuracy on test set: 0.900


### Comparison between different kind of datasets

In [13]:
perf('../dumps/control_8000_false_3.csv','mlp1',False)

Accuracy on training set: 0.900
Accuracy on test set: 0.897


#### Different thresholds

In [14]:
perf('../dumps/default_1.csv','mlp1',False)

Accuracy on training set: 0.783
Accuracy on test set: 0.771


In [15]:
perf('../dumps/default_2.csv','mlp1',False)

Accuracy on training set: 0.854
Accuracy on test set: 0.843


In [16]:
perf('../dumps/default_4.csv','mlp1',False)

Accuracy on training set: 0.943
Accuracy on test set: 0.948


In [17]:
perf('../dumps/default_5.csv','mlp1',False)

Accuracy on training set: 0.943
Accuracy on test set: 0.948


#### Errors considered as positive result from detector

In [18]:
perf('../dumps/error_as_packed.csv','mlp1',False)

Accuracy on training set: 0.853
Accuracy on test set: 0.843


#### Check influence of each detector

In [19]:
perf('../dumps/not_die.csv','mlp1',False)

Accuracy on training set: 0.928
Accuracy on test set: 0.929


In [20]:
perf('../dumps/not_cisco.csv','mlp1',False)

Accuracy on training set: 0.918
Accuracy on test set: 0.918


In [21]:
perf('../dumps/not_manalyze.csv','mlp1',False)

Accuracy on training set: 0.941
Accuracy on test set: 0.946


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [22]:
perf('../dumps/not_peid.csv','mlp1',False)

Accuracy on training set: 0.943
Accuracy on test set: 0.948


In [23]:
perf('../dumps/not_peframe.csv','mlp1',False)

Accuracy on training set: 0.943
Accuracy on test set: 0.947


NO CONVERGENCE when we remove manalyze, quite interesting !

#### Only boolean values

In [2]:
perf('../dumps/control_8000_false_3.csv','mlp1',True)

Most important : 

Accuracy on training set: 0.900
Accuracy on test set: 0.897
---------------

All features : 

Accuracy on training set: 0.900
Accuracy on test set: 0.897


### Features relevance

In [6]:
gt = pd.read_csv('../dumps/2020.02.10-12.14.csv')
cols = [col for col in gt.columns if col not in ['label']]
data = gt[cols]
target = gt['label']

data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.20, random_state = 0)
scaler = Normalizer()
scaler.fit(data_train)
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)

mlp = MLPClassifier()
mlp.fit(data_train, target_train)

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [7]:
feature_selection('../dumps/2020.02.10-12.14.csv',0.15,"mlp")

0.15
(1196, 119)


AttributeError: 'str' object has no attribute 'fit'

### Test with Thomas datasets

In [8]:
gt = pd.read_csv("../dumps/2019-08.Merged_thomas.csv")
cols = [col for col in gt.columns if col not in ['label']]
data = gt[cols]
target = gt['label']

data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.20, random_state = 0)
scaler = Normalizer()
scaler.fit(data_train)
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)

tree = MLPClassifier(solver='adam',activation='tanh',alpha=100,hidden_layer_sizes=(50, 50, 100))
tree.fit(data_train, target_train)
print("Accuracy on training set: {:.3f}".format(tree.score(data_train, target_train))) 
print("Accuracy on test set: {:.3f}".format(tree.score(data_test, target_test)))

Accuracy on training set: 0.769
Accuracy on test set: 0.761


In [9]:
gt = pd.read_csv("../dumps/2019-09.Merged_thomas.csv")
cols = [col for col in gt.columns if col not in ['label']]
data = gt[cols]
target = gt['label']

data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.20, random_state = 0)
scaler = Normalizer()
scaler.fit(data_train)
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)

tree = MLPClassifier(solver='adam',activation='tanh',alpha=100,hidden_layer_sizes=(50, 50, 100))
tree.fit(data_train, target_train)
print("Accuracy on training set: {:.3f}".format(tree.score(data_train, target_train))) 
print("Accuracy on test set: {:.3f}".format(tree.score(data_test, target_test)))

Accuracy on training set: 0.784
Accuracy on test set: 0.787


### PCA

In [16]:
PCA_reduction('../dumps/2020.03.11-17.39.csv','mlp1')

  Variance    Training acc    Test acc    Components    Time (s)
----------  --------------  ----------  ------------  ----------
      1           0.858357    0.857531           119    14.8017
      0.99        0.858357    0.857531            98     9.46293
      0.95        0.858357    0.857531            77     8.73768
      0.9         0.858357    0.857531            60     8.37715
      0.85        0.858357    0.857531            47     8.2692


This test was conducted with the following caracteristics:
- MLPClassifier(solver='adam',activation='tanh',alpha=100,hidden_layer_sizes=(50, 50, 100))

which were obtained by selecting the best results from the previous tests before normalization. As we can see, overall accuracies are not really satisfying. We can also observe what we can observe for nearly every PCA, pretty same accuracies with less components in half the time.

In [19]:
PCA_reduction('../dumps/2020.03.11-17.39.csv','mlp2')

  Variance    Training acc    Test acc    Components    Time (s)
----------  --------------  ----------  ------------  ----------
      1           0.992348    0.978239           119     84.9651
      0.99        0.992263    0.978239            98     72.8201
      0.95        0.991583    0.977899            77     48.1287
      0.9         0.991498    0.976879            60     74.0656
      0.85        0.989628    0.979259            47     42.5804


The parameters here were chosen after normalization leading to this configuration : 
- MLPClassifier(solver='sgd',activation='tanh',alpha=0.1,hidden_layer_sizes=(100,50,50),max_iter=1000)

which offer really good performance and the training set for less components and half computation time.

### Time fiability

In [9]:
gt = pd.read_csv("../dumps/default_20190615_6000.csv")
cols = [col for col in gt.columns if col not in ['label']]
data_train = gt[cols]
target_train = gt['label']

scaler = Normalizer()
scaler.fit(data_train)
data_train = scaler.transform(data_train)

clf = MLPClassifier(solver='sgd',activation='tanh',alpha=0.1,hidden_layer_sizes=(100,50,50),max_iter=1000)
clf.fit(data_train, target_train)
dump(clf,"snapshots/mlp_default_20190615_6000.joblib")

['snapshots/mlp_default_20190615_6000.joblib']

In [10]:
clf = load("snapshots/mlp_default_20190615_6000.joblib")

gt = pd.read_csv("../dumps/default_20190808_1000.csv")
cols = [col for col in gt.columns if col not in ['label']]
data_test = gt[cols]
target_test = gt['label']

data_test = scaler.transform(data_test)

print("Accuracy on training set: {:.3f}".format(clf.score(data_train, target_train))) 
print("Accuracy on test set: {:.3f}".format(clf.score(data_test, target_test)))

Accuracy on training set: 0.919
Accuracy on test set: 0.897


Conclusion : we can assess that performances are kept in a reasonable range over time.

#### Long run

In [2]:
time_comparison('mlp1')

Acceptation threshold : 1/5 

  # malwares in training set    Approx. period in weeks    Training acc    Test acc
----------------------------  -------------------------  --------------  ----------
                        6000                   0.857143        0.7965         0.691
                       14000                   2               0.782857       0.691
                       21000                   3               0.754286       0.691
                       31000                   4.42857         0.689258       0.691
Acceptation threshold : 2/5 

  # malwares in training set    Approx. period in weeks    Training acc    Test acc
----------------------------  -------------------------  --------------  ----------
                        6000                   0.857143        0.868833       0.855
                       14000                   2               0.856286       0.855
                       21000                   3               0.827905       0.855
                

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


  # malwares in training set    Approx. period in weeks    Training acc    Test acc
----------------------------  -------------------------  --------------  ----------
                        6000                   0.857143        0.953167        0.94
                       14000                   2               0.947786        0.94
                       21000                   3               0.938952        0.94
                       31000                   4.42857         0.938935        0.94
Acceptation threshold : 5/5 



/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


  # malwares in training set    Approx. period in weeks    Training acc    Test acc
----------------------------  -------------------------  --------------  ----------
                        6000                   0.857143        0.953167        0.94
                       14000                   2               0.947786        0.94
                       21000                   3               0.938952        0.94
                       31000                   4.42857         0.938935        0.94


In [ ]:
time_comparison('mlp2')

Acceptation threshold : 1/5 

  # malwares in training set    Approx. period in weeks    Training acc    Test acc
----------------------------  -------------------------  --------------  ----------
                        6000                   0.857143        0.821333       0.709
                       14000                   2               0.799571       0.72
                       21000                   3               0.801667       0.795
                       31000                   4.42857         0.761839       0.807
Acceptation threshold : 2/5 



/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Same conclusion as linear models, by increasing the training set, the performances on the test set do not necessarily improve, it's quite constant.